# TF-IDF

In [15]:
import pandas as pd
import numpy as np

from pandas.errors import EmptyDataError

# 정규식 활용 -> 특수문자 제거
import re

# 한글 형태소 분석기
from konlpy.tag import Okt
okt = Okt()

# keybert
from keybert import KeyBERT

In [16]:
date_str = '20250819'

In [17]:
def safe_read_csv(path):
    try:
        return pd.read_csv(path)
    except EmptyDataError:
        print(f"⚠️ {path} 는 비어있음 → 스킵")
        return pd.DataFrame()   # 빈 DF 반환
    except FileNotFoundError:
        print(f"❌ {path} 파일 없음 → 스킵")
        return pd.DataFrame()

# 파일 불러오기
raw_data_i = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_i_{date_str}.csv")
raw_data_g = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_g_{date_str}.csv")
raw_data_gen_1 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_00000-19999.csv")
raw_data_gen_2 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_20000-39999.csv")
raw_data_gen_3 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_40000-59999.csv")
raw_data_gen_4 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_60000-79999.csv")
raw_data_gen_5 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_80000-99999.csv")

# 하나의 파일로 유니온 (비어있으면 자동 무시)
raw_data = pd.concat([
    raw_data_gen_1, raw_data_gen_2, raw_data_gen_3,
    raw_data_gen_4, raw_data_gen_5,
    raw_data_i, raw_data_g
], ignore_index=True)

print("총 데이터 크기:", raw_data.shape)

# 완전일치 중복기사 제거
df = raw_data.drop_duplicates(subset=['title', 'text'], keep='first').copy()

# title에 [속보] 또는 [포토] 포함 시 text를 NaN으로 변경
df.loc[df['title'].str.contains(r'\[속보\]|\[포토\]', regex=True), 'text'] = np.nan

# null셀을 빈칸으로 만들기
df['title'] = df['title'].fillna('')
df['text'] = df['text'].fillna('')

# 특수문자 등 전처리
df['text_clean'] = df['text'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df['title_clean'] = df['title'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df.head(3)

⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250819_00000-19999.csv 는 비어있음 → 스킵
⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250819_20000-39999.csv 는 비어있음 → 스킵
⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250819_40000-59999.csv 는 비어있음 → 스킵
총 데이터 크기: (619, 4)


,url,title,text,publish_date,text_clean,title_clean
0,https://www.hankyung.com/article/2025081970714,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaN,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다 회담은 뉴욕 증시가 마감할 무렵 ...,한국경제
2,https://www.hankyung.com/article/2025081971696,"""'불닭' 따라잡기 쉽지 않네""…농심, 해외사업 '시름' [종목+]",미국·중국 판매 채널 확장 비용에 수익성 악화\n\n2분기 실적 분석 증권사 12곳...,2025-08-19 07:40:33+09:00,미국 중국 판매 채널 확장 비용에 수익성 악화 분기 실적 분석 증권사 곳 중...,불닭 따라잡기 쉽지 않네 농심 해외사업 시름 종목
3,https://www.hankyung.com/article/2025081971727,"쌍문동 아파트 화재, 주민 170여명 대피…소방대원 부상",한밤중 화재에 주민들 대피\n\n약 1시간 만에 화재 진압\n\n이미지=연합뉴스\n...,2025-08-19 06:31:34+09:00,한밤중 화재에 주민들 대피 약 시간 만에 화재 진압 이미지 연합뉴스 서울 도...,쌍문동 아파트 화재 주민 여명 대피 소방대원 부상


In [33]:
# 불용어제거 -> 형태소 조사 삭제 -> tf-idf -> 가장 중요한 10개 단어 뽑아내기

In [18]:
# 불용어 불러오기 (줄바꿈 기준 분리)
with open("/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/stopwords_kor.txt", "r", encoding="utf-8") as f:
    stopwords = f.read()

# 불용어 제거 함수
def remove_stopwords(text_clean):
    if not isinstance(text_clean, str):
        return ''
    return ' '.join([word for word in text_clean.split() if word not in stopwords])

# 조사 제거 함수
def remove_josa(text):
    if not isinstance(text, str):
        return ''
    return ' '.join([word for word, pos in okt.pos(text) if pos != 'Josa'])

# 조사 제거 → 불용어 제거 순차 적용
df['title_clean'] = df['title_clean'].apply(remove_josa)  # 조사 제거
df['title_clean'] = df['title_clean'].apply(remove_stopwords)  # 불용어 제거

df['text_clean'] = df['text_clean'].apply(remove_josa)  # 조사 제거
df['text_clean'] = df['text_clean'].apply(remove_stopwords)  # 불용어 제거

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1) 코퍼스 준비 (NaN 방지)
corpus = df['text_clean'].fillna('')

# 2) 벡터라이저 설정
# - tokenizer=lambda s: s.split(): 이미 공백으로 토큰화된 텍스트를 그대로 사용
# - lowercase=False: 한글/대소문 유지
# - ngram_range=(1,1): 단어 단위(필요 시 (1,2) 등으로 확장)
# - min_df, max_df, max_features는 데이터 규모에 맞게 조절
vectorizer = TfidfVectorizer(
    tokenizer=lambda s: s.split(),
    token_pattern=None,
    lowercase=False,
    ngram_range=(1,1),
    min_df=2,           # 2개 미만 문서에만 나온 단어 제거
    max_df=0.7,         # 70% 넘는 문서에 등장하는 흔한 단어 제거
    max_features=8000   # 상위 8천개 단어만 사용 (메모리 절약용)
)

# 3) 학습 및 변환 (희소행렬)
X = vectorizer.fit_transform(corpus)   # shape: (문서수, 단어수)
terms = vectorizer.get_feature_names_out()

# 4) (옵션) DataFrame으로 시각화 — 메모리 주의
# tfidf_df = pd.DataFrame.sparse.from_spmatrix(X, columns=terms)

# 5) 각 문서별 TF-IDF 상위 키워드 뽑기 함수
def top_tfidf_terms(row_index: int, topk: int = 10):
    row = X.getrow(row_index).toarray().ravel()
    if not np.any(row):
        return []
    idx = np.argsort(row)[::-1][:topk]
    return list(zip(terms[idx], row[idx]))

# 6) 모든 문서에 대해 상위 키워드 뽑아 df에 붙이기 (리스트→문자열)
topk = 10
df['tfidf_top_terms'] = [
    ', '.join([f'{t}:{w:.3f}' for t, w in top_tfidf_terms(i, topk)]) 
    for i in range(X.shape[0])
]

In [20]:
df.head()

,url,title,text,publish_date,text_clean,title_clean,tfidf_top_terms
0,https://www.hankyung.com/article/2025081970714,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaN,트럼프 대통령 푸틴 러시아 대통령 만났습니다 회담 뉴욕 증시 마감 관망 나타났습니다...,한국,"관세:0.446, 인플레이션:0.234, 물가:0.212, 금리:0.204, 파월:..."
2,https://www.hankyung.com/article/2025081971696,"""'불닭' 따라잡기 쉽지 않네""…농심, 해외사업 '시름' [종목+]",미국·중국 판매 채널 확장 비용에 수익성 악화\n\n2분기 실적 분석 증권사 12곳...,2025-08-19 07:40:33+09:00,미국 중국 판매 채널 확장 비용 수익 악화 실적 증권사 목표 하향 삼양식품 실적 미...,불닭 따라잡기 쉽지 않네 농심 해외 사업 시름 종목,"농심:0.341, 실적:0.340, 증권:0.291, 목표:0.217, 영업:0.2..."
3,https://www.hankyung.com/article/2025081971727,"쌍문동 아파트 화재, 주민 170여명 대피…소방대원 부상",한밤중 화재에 주민들 대피\n\n약 1시간 만에 화재 진압\n\n이미지=연합뉴스\n...,2025-08-19 06:31:34+09:00,한밤중 화재 주민 대피 화재 진압 이미지 연합뉴스 서울 도봉구 쌍문동 아파트 한밤중...,쌍문동 아파트 화재 주민 여명 대피 소방대 부상,"화재:0.484, 주민:0.453, 대피:0.414, 다친:0.265, 아파트:0...."
4,https://www.hankyung.com/article/2025081971787,'金치' 먹게 생겼네…배추 한포기 사고 1만원 내면 거스름돈이,폭염·폭우 이어지더니\n\n배추 한포기 7000원 넘겨\n\n서울 동대문구 경동시장...,2025-08-19 06:45:21+09:00,폭염 폭우 이어지더니 배추 포기 넘겨 서울 동대문구 채소 배추 진열 뉴스 폭염 폭우...,먹게 생겼네 배추 포기 사고 내면 거스름 돈,"배추:0.558, 가격:0.498, 평년:0.248, 작년:0.240, 도매:0.1..."
5,https://www.hankyung.com/article/2025081971797,"""젤렌스키 정장 멋지다"" 확 달라진 트럼프…회담장 '화기애애' [영상]",트럼프·젤렌스키 회담\n\n2월 정상회담과 같은 '파국' 없어\n\n영상 모듈 닫기...,2025-08-19 07:04:27+09:00,트럼프 젤 렌스 회담 정상회담 파국 영상 모듈 닫기 AP EPA 미국 워싱턴 DC ...,젤 렌스 정장 멋지다 달라진 트럼프 회담 화기 애애 영상,"대통령:0.463, 렌스:0.441, 젤:0.436, 회담:0.294, 트럼프:0...."


In [21]:
from collections import Counter
import re

# 1. 전체 키워드 모으기 (점수 제거)
all_terms = []
for row in df['tfidf_top_terms']:
    if not isinstance(row, str):
        continue
    # "단어:점수" → 단어만 추출
    terms = [re.split(r':', t)[0] for t in row.split(', ')]
    all_terms.extend(terms)

# 2. 빈도수 계산
term_freq = Counter(all_terms)

# 3. DataFrame으로 변환
term_freq_df = pd.DataFrame(term_freq.items(), columns=['term', 'count'])
term_freq_df = term_freq_df.sort_values(by='count', ascending=False).reset_index(drop=True)

term_freq_df.head(10)

,term,count
0,,34
1,대통령,25
2,정부,18
3,미국,17
4,중국,16
5,우크라이나,15
6,매출,15
7,AI,15
8,트럼프,15
9,사업,14


In [23]:
term_freq_df.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/text_TFIDF.csv", index=False)